In [1]:
import dotenv
dotenv.load_dotenv()
import openai
import os

In [2]:
api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=api_key)

In [3]:
import pandas as pd
import evaluate
from transformers import pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import DataFrameLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_openai import ChatOpenAI
import os
import getpass
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from langchain.llms import HuggingFaceHub

/usr/local/Caskroom/miniconda/base/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
query = 'What classes does the boating center offer to community members?'

In [38]:
context = """
UC Santa Cruz Community Boating Center is a community-based boating program that connects students and community members to the Monterey Bay through recreational activities including sailing, rowing and kayaking.
UC Santa Cruz Community Boating Center staff and veteran participants provide a familiarity with the equipment, skills, and water safety that comes from a lifetime of enjoying these sports.
The Community Boating Center also connects people who are interested in these sports to one another, whether you are a beginner seeking to learn something new or a experienced sailor, rower, and/or kayaker looking to improve your skills and get time on the water. Whether you've never been on a boat before, or want to build on old skills, we have a class for you! From 2-week junior summer camps to quarterly physical education classes for UC Santa Cruz Students, you'll be cruising around Monterey Bay with ease.

Our club exists to give community members, UC Santa Cruz students, faculty, and staff access to equipment that makes it possible to enjoy our beautiful Monterey Bay. The club provides members a unique opportunity to use any number of sailing and rowing vessels under the weekend supervision of the Boating Center dockmaster.
Our boats:
Sailing vessels for weekend use currently include RS Quests, Lasers, RS Visions, a SC27, a Santana 22, a J 24, an Olson 30, and an O'Day 34.
The rowing fleet exists of eight Edon training shells, two MAAS AEROS 20s, two MAAS 24s, Bay 21, Pegasus, WinTech 30' double, and a custom-built wooden double. 
Want to take a boat out? Make sure to pay your membership dues and boat fees online ahead of time. The boating center will no longer be accepting cash payments on-site.
Sailing Hours:
Friday 4:00-7:00 PM (during daylight savings)
Saturday 12:00-5:00 PM
Sunday 12:00-5:00 PM
Members of the Community Boating Club have access to sailboats and rowing shells during club hours. Members must be approved for boat use by the boating directors before checking out a rowing shall or sailboat. Check our calendar for the next available checkout day. If you do not have prior sailing or rowing experience, we offer classes to build your confidence on the water! For class information, click here.
Sailors and rowers are evaluated based on experience and allowed boat use accordingly.

Boating Club Hours
The Community Boating Center is open 2 days a week.
Saturday: 12:00 PM - 5:00 PM
Sunday: 12:00 PM - 5:00 PM

How can I join the Boating Club?
"""

In [49]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": f"Given this query and context, return me a prolog program to evaluate. Only respond in prolog. Query: {query}, Context: {context}"}
    ]
)
prolog = response.choices[0].message.content
print(prolog)

classes_offered_by_boating_center(Class) :-
    member(Class, ['sailing', 'rowing', 'kayaking']).

eval(Day, Member, Experience, Class) :-
    class_offered_by_boating_center(Class),
    member(Day, ['Saturday', 'Sunday']),
    member(Times, ['12:00 PM - 5:00 PM']),
    member(Member, ['community member', 'UC Santa Cruz student', 'faculty', 'staff']),
    member(Experience, ['beginner', 'experienced']),
    member(Evaluation, ['approved by boating directors']). 

join_boating_club(Person) :-
    member(Person, ['community member', 'UC Santa Cruz student', 'faculty', 'staff']),
    pay_membership_dues(Person),
    pay_boat_fees(Person),
    check_calendar(Person),
    evaluate_experience(Person), 
    approved_for_boat_use(Person).

pay_membership_dues(Member) :-
    member(Member, ['community member', 'UC Santa Cruz student', 'faculty', 'staff']).

pay_boat_fees(Member) :-
    member(Member, ['community member', 'UC Santa Cruz student', 'faculty', 'staff']).

check_calendar(Member) :-


In [40]:
def strip_markdown_code_block(text):
    if text.startswith("```"):
        lines = text.strip().splitlines()
        # Remove first and last lines (e.g. ```prolog and ```)
        return "\n".join(lines[1:-1])
    return text

In [44]:
clean_prolog = """
classes_offered_by_boating_center(Class) :-
    member(Class, ['sailing', 'rowing', 'kayaking']).
"""
print(clean_prolog)


classes_offered_by_boating_center(Class) :-
    member(Class, ['sailing', 'rowing', 'kayaking']).



In [45]:
with open("rules.pl", "w") as f:
    f.write(clean_prolog)

In [46]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": f"Here I have a prolog rule and query, what is the prolog query I should run? Query: {query}, Rulebase: {clean_prolog} Only output the prolog query. "}
    ]
)
prolog_query = response.choices[0].message.content
print(prolog_query)

classes_offered_by_boating_center(Class).


In [31]:
def clean_query(query):
    return query[3:]

c_query = clean_query(prolog_query)
print(c_query)

boating_classes.


In [57]:
import subprocess

# File containing Prolog code
prolog_file = "rules.pl"

# This assumes your .pl file defines a predicate called `query/0`
result = subprocess.run(
    ["swipl", "-s", prolog_file, "-g", "query"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

print("STDOUT:")
print(result.stdout)
print("STDERR:")
print(result.stderr)


STDOUT:
pizza

STDERR:



In [ ]:
from pyswip import Prolog

prolog = Prolog()
prolog.consult("rules.pl")

# Run a query
results = list(prolog.query(c_query))

# Print the results
for result in results:
    print(result)
